In [0]:
import pandas as pd
import numpy as np

pth='E:/Documents/Educacion/Maestría/UP_DataScience/MachineLearning/insider_dwh/'
train_x = pd.read_csv(pth + "ins_train_x_resampled.csv")
train_y = pd.read_csv(pth + "ins_train_y_resampled.csv")
#dropping period
train_x.drop(["period"], axis = 1, inplace = True)

In [0]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = list(range(10,100,10))
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = list(range(5,10,2))
#max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [0]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 2020)

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_x, train_y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   25.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.1min finished
C:\Users\ulise\Anaconda3\envs\master_ML\lib\site-packages\sklearn\model_selection\_search.py:714: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [0]:
rf_random.best_params_


print("Best estimator found by grid search:")
print(rf_random.best_estimator_)

Best estimator found by grid search:
RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=9, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=2020,
                       verbose=0, warm_start=False)


In [0]:
model = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=9, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=2020,
                       verbose=0, warm_start=False)
model.fit(train_x, np.ravel(train_y))

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=9, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=2020,
                       verbose=0, warm_start=False)

In [0]:
test_x = pd.read_csv(pth + "ins_test_x.csv")
test_y = pd.read_csv(pth + "ins_test_y.csv", header=None)

test_x.drop(["period"], axis = 1, inplace = True)

In [0]:
y_pred = model.predict(test_x)

In [0]:
from sklearn.metrics import classification_report

print(classification_report(test_y,y_pred))

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      3775
         1.0       0.98      0.80      0.88       418

    accuracy                           0.98      4193
   macro avg       0.98      0.90      0.94      4193
weighted avg       0.98      0.98      0.98      4193



In [0]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(test_y, y_pred)
print(confusion_matrix)

[[3769    6]
 [  83  335]]


In [0]:
#out of time test
insider_oot = pd.read_csv(pth + "insider_OOT.csv")
insider_oot.head()

test_x_oot = insider_oot.drop(["period", "isInsider"], axis = 1)
test_y_oot = insider_oot["isInsider"]
test_y_oot=test_y_oot.to_numpy()
y_pred_oot = model.predict(test_x_oot)

print(classification_report(test_y_oot,y_pred_oot))

              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99      2557
         1.0       0.52      0.97      0.67        35

    accuracy                           0.99      2592
   macro avg       0.76      0.98      0.83      2592
weighted avg       0.99      0.99      0.99      2592



In [0]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(test_y_oot.tolist(),y_pred_oot.tolist())
print(confusion_matrix)

[[2525   32]
 [   1   34]]


In [0]:
import pickle
mdlpth = 'E:/Documents/Educacion/Maestría/UP_DataScience/MachineLearning/models/'
pickle.dump(model, open(mdlpth + "RF_Insider", 'wb'))
